# Modelado de proteínas y preparación de péptidos
Tras la prediccion de los potenciales epítopos, y la seleccion de los MHC con los que van a interactuar, se procedió con el modelado de la estructura 3D de ambos.

## 1. Modelado de las estructuras 3D de los alelos MHC
Las estructuras 3D de la mayoría de los alelos MHC se extrayeron de la base de datos '**RCSB Protein Data Bank**' - RCSB PDB (https://www.rcsb.org/) (*Berman et al., 2000*), siendo estos los alelos HLA-A*01:01 (PDB ID: 6MPP), HLA-A*02:01 (PDB ID: 4U6X), HLA-A*03:01 (PDB ID: 3RL1), HLA-A*24:02 (PDB ID: 7JYV), HLA-B*35:01 (PDB ID: 4PRN), HLA-B*44:03 (PDB ID: 3DX7), HLA-B*51:01 (PDB ID: 1E28), HLA-C*07:02 (PDB ID: 5VGE) para los MHC de clase I; y HLA-DRB1*07:01 (PDB ID: 7Z0Q), HLA-DRB1*14:02 (PDB ID: 6ATF) para los MHC de clase II. 

In [1]:
import os
import urllib
import sys

In [2]:
def download_pdb(pdbcode, datadir, downloadurl = 'https://files.rcsb.org/download/'):
    pdbfn = pdbcode + '.pdb'
    url = downloadurl + pdbfn
    outfnm = os.path.join(datadir, pdbfn)
    try:
        urllib.request.urlretrieve(url, outfnm)
        return outfnm
    except Exception as err:
        print(str(err), file = sys.stderr)
        return None

In [17]:
ID = ['6MPP', '4U6X', '3RL1', '7JYV', '4PRN', '3DX7', '1E28', '5VGE', '7Z0Q', '6ATF']

for ID_PDB in ID:
    download_pdb(ID_PDB, '/home/victor/Escritorio/Tesis/RESULTADOS/9. Modelado de proteínas y preparación de péptidos/PDB_RCSB')
    print(f'{ID_PDB}.pdb was downloaded')

6MPP.pdb was downloaded
4U6X.pdb was downloaded
3RL1.pdb was downloaded
7JYV.pdb was downloaded
4PRN.pdb was downloaded
3DX7.pdb was downloaded
1E28.pdb was downloaded
5VGE.pdb was downloaded
7Z0Q.pdb was downloaded
6ATF.pdb was downloaded


En el caso de los alelos HLA-A*31:01, HLA-C*03:04, HLA-C*04:01, HLA-C*07:01, HLA-DRB1*03:01, HLA-DRB1*04:07 y HLA-DRB1*08:02, fueron generados por modelado de homología a través de un *in house* script de Python basado en la API de **SWISS-MODEL** (https://swissmodel.expasy.org/) (*Waterhouse et al., 2018*). Para ello, las secuencias aminoacídicas de dichos alelos fueron tomadas de la base de datos **IPD-IMGT/HLA** (https://www.ebi.ac.uk/ipd/imgt/hla/) (*Barker et al., 2023*). Se comprobaron los parámetros de estimación de calidad, como las puntuaciones *Qmean* y el porcentaje de identidad de las estructuras de los alelos MHC modelados. Alternativamente, las estructuras 

In [3]:
# Para conseguir las estructuras 3D de los 7 HLA en mención, se utilizó Swiss-Model

# Diseñar un archivo de entrada exclusivamente con los nombres de los HLA
file_name = open('HLA_secuencias.fasta')

## Nombres de cada proteína
data_name = []
for pos_name, l_num_name in enumerate(file_name):
    # comprobar si el número de línea se especifica en las líneas para leer la matriz
    if pos_name %3 == 0:
        # imprimir el número de línea requerido
        data_name.append(l_num_name)
        
each_name = [y.rstrip('\n') for y in data_name]

In [5]:
# Diseñar un archivo de entrada exclusivamente con las secuencias de la cadena alfa de los HLA
file_seq_A = open('HLA_secuencias.fasta')

## Secuencias aminoacídicas de cada proteína
data_seq_A = []
for pos_A, l_num_A in enumerate(file_seq_A):
    # comprobar si el número de línea se especifica en las líneas para leer la matriz
    if pos_A %3 == 1:
        # imprimir el número de línea requerido
        data_seq_A.append(l_num_A)

each_seq_A = [x.rstrip('\n') for x in data_seq_A]

In [6]:
# Diseñar un archivo de entrada exclusivamente con las secuencias de la cadena beta de los HLA
file_seq_B = open('HLA_secuencias.fasta')

## Secuencias aminoacídicas de cada proteína
data_seq_B = []
for pos_B, l_num_B in enumerate(file_seq_B):
    # comprobar si el número de línea se especifica en las líneas para leer la matriz
    if pos_B %3 == 2:
        # imprimir el número de línea requerido
        data_seq_B.append(l_num_B)

each_seq_B = [x.rstrip('\n') for x in data_seq_B]

In [7]:
# Importar el módulo 
import requests
import time

In [7]:
# Utilizar Swiss-Model para obtener las estructuras 3D modeladas de los HLA objetivo:
for name, sequence_A, sequence_B in zip(each_name, each_seq_A, each_seq_B):
    response = requests.post(
        'https://swissmodel.expasy.org/automodel',
        headers = { 'Authorization': f'Token 8f40c9582d7e76d9debe518204bd24316aa2afad' }, # Insertar el token de mi cuenta
        json = { 
            'target_sequences': 
                [
                    f'{sequence_A}',
                    f'{sequence_B}'
                ],
            'project_title':f'This is an modelling of the {name}'
          })

    project_id = response.json()['project_id']
    
    while True:
        time.sleep(10)

        # Actualizar el estado desde el servidor
        response = requests.get(
            f'https://swissmodel.expasy.org/project/{ project_id }/models/summary/', 
            headers = { 'Authorization': f'Token 8f40c9582d7e76d9debe518204bd24316aa2afad' })

        status = response.json()['status']
            
        if status in ['COMPLETED', 'FAILED']:
            break    
            print('Job status is now', status)        

    response_object = response.json()
    if response_object['status'] == 'COMPLETED':
        for model in response_object['models']:
            print(f'{name} has this result:', model['coordinates_url'])
        print('...........')

>HLA-A31:01 has this result: https://swissmodel.expasy.org/project/3233ca/models/01.pdb.gz
...........
>HLA-C03:04 has this result: https://swissmodel.expasy.org/project/ebac90/models/01.pdb.gz
...........
>HLA-C04:01 has this result: https://swissmodel.expasy.org/project/d6233b/models/01.pdb.gz
...........
>HLA-C07:01 has this result: https://swissmodel.expasy.org/project/f78fe5/models/01.pdb.gz
...........
>HLA-DRB103:01 has this result: https://swissmodel.expasy.org/project/1b7394/models/01.pdb.gz
...........
>HLA-DRB104:07 has this result: https://swissmodel.expasy.org/project/7bc3f6/models/01.pdb.gz
...........
>HLA-DRB108:02 has this result: https://swissmodel.expasy.org/project/4f1738/models/01.pdb.gz
...........


Luego, todos los HLA fueron preparados con un script en Python basado en **PDBFixer** para evitar átomos perdidos y residuos no estándar: https://colab.research.google.com/drive/12M49pviNea9TqMQMFjziU7iSakXsDbH9?usp=sharing

## 2. Predicción de las estructuras terciarias de los epítopos seleccionados
Las estructuras terciarias o 3D de los mejores epítopos seleccionados se generaron utilizando la herramienta **PEP-FOLD 3** (https://bioserv.rpbs.univ-paris-diderot.fr/services/PEP-FOLD3/).

In [26]:
# Instalar los paquetes 'selenium' y 'biopython'
!pip install selenium
!pip install biopython

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [23]:
# Importar los módulos necesarios de 'selenium', 'Bio' y 'collections'
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import time

import Bio
import collections
from Bio import SeqIO

In [10]:
# Establecer la ubicación del WebDriver
s = Service('/usr/local/bin/chromedriver')

In [40]:
# Inicializar el Selenium WebDriver
driver = webdriver.Chrome(service = s)

# Visitar la herramienta PEP-FOLD 3
driver.get('https://mobyle.rpbs.univ-paris-diderot.fr/cgi-bin/portal.py#forms::PEP-FOLD3')
time.sleep(4)

# Definir las secuencias aminoacídicas de los epítopos
epítopos = '/home/victor/Escritorio/Tesis/RESULTADOS/9. Modelado de proteínas y preparación de péptidos/Lista_epitopos.fasta'

# Editar los parámetros
## Eliminar las secuencias
clear = driver.find_element(by = By.XPATH, value = '/html/body/table[2]/tbody/tr/td[3]/div/div[2]/div/div/div/form/fieldset[1]/div/fieldset/div[2]/ul/li[5]/button[2]')

## Insertar el epítopo
epitope_seq = driver.find_element(by = By.XPATH, value = '/html/body/table[2]/tbody/tr/td[3]/div/div[2]/div/div/div/form/fieldset[1]/div/fieldset/div[2]/div/div[1]/div/textarea')


# Realizar la predicción de antigenicidad de los epítopos
for rec in SeqIO.parse(epítopos,'fasta'):
    seq_list = rec.seq
    
    clear.click()
    epitope_seq.send_keys(f'{seq_list}')
    # Entregar la data
    driver.find_element(by = By.XPATH, value = '/html/body/table[2]/tbody/tr/td[3]/div/div[2]/div/div/div/form/table/tbody/tr/td[2]/span/input[1]').click()
    time.sleep(20)
    # Regresar al inicio
    get_url = driver.current_url
    driver.find_element(by = By.XPATH, value = '/html/body/table[2]/tbody/tr/td[3]/div/div[4]/div/div[2]/div/div/fieldset[1]/a[3]/button').click()
    time.sleep(6)
    
    driver.get('https://mobyle.rpbs.univ-paris-diderot.fr/cgi-bin/portal.py#forms::PEP-FOLD3')
    time.sleep(4)
    epítopos = '/home/victor/Escritorio/Tesis/RESULTADOS/9. Modelado de proteínas y preparación de péptidos/Lista_epitopos.fasta'
    clear = driver.find_element(by = By.XPATH, value = '/html/body/table[2]/tbody/tr/td[3]/div/div[2]/div/div/div/form/fieldset[1]/div/fieldset/div[2]/ul/li[5]/button[2]')
    epitope_seq = driver.find_element(by = By.XPATH, value = '/html/body/table[2]/tbody/tr/td[3]/div/div[2]/div/div/div/form/fieldset[1]/div/fieldset/div[2]/div/div[1]/div/textarea')


print('All epitopes were generated!')

All epitopes were generated!


## 3. Visualización 3D de los epítopos

Las estructuras en 3D de los epítopos se visualizaron con el visor molecular **PyMOL v2.5** (https://pymol.org/2/) (*Schrodinger, 2010*).  ̈

In [1]:
# Instalar el paquete 'nglview'
!pip install py3Dmol

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import py3Dmol

In [2]:
# CTL_NS3 (TVWFVPSIK)
view = py3Dmol.view()
view.addModel(open('Epítopos/CTL_NS3_TVWFVPSIK.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'skyblue'}})
view.zoomTo()
view.setBackgroundColor('white')
view.show()
# view.png()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
# CTL_NS3 (TVWFVPSIK)
view = py3Dmol.view()
view.addModel(open('Epítopos/CTL_NS5_MYFHRRDLR.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'yellow'}})
view.zoomTo()
view.setBackgroundColor('white')
view.show()
# view.png()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
# HTL_NS1 (RAVHADMGYWIE)
view = py3Dmol.view()
view.addModel(open('Epítopos/HTL_NS1 _RAVHADMGYWIE.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'red'}})
view.zoomTo()
view.setBackgroundColor('white')
view.show()
#view.png()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
# HTL_NS3_1 (WDFVVTTDISEM)
view = py3Dmol.view()
view.addModel(open('Epítopos/HTL_NS3_1_WDFVVTTDISEM.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'purple'}})
view.zoomTo()
view.setBackgroundColor('white')
view.show()
# view.png()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
# HTL_NS3_2 (DWDFVVTTDISE)
view = py3Dmol.view()
view.addModel(open('Epítopos/HTL_NS3_2_DWDFVVTTDISE.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'orange'}})
view.zoomTo()
view.setBackgroundColor('white')
view.show()
# view.png()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol